# Movie Recommendation AI Ising Model

### Testing Parameters

In [10]:
USER_COUNT: int = 1000          # Number of users
MOVIE_COUNT: int = 250          # Number of movies
C_VALUE: float = 0.045          # Regularization constant
MINIMUM_RATING: float = 4.0  # Minimum rating to classify movie as liked

### Start of Code

In [11]:
import os
import zipfile
import urllib.request
import ssl
import pandas as pd
import numpy as np
import pathlib as p
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datetime import datetime

# pyGMs library
import pyGMs as gm
import pyGMs.ising
import pyGMs.wmb

# removes deprecation warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


### Downloading the Dataset

In [13]:
dataset_url: str = "https://files.grouplens.org/datasets/movielens/ml-latest.zip"
zip_file: str = "ml-latest.zip"
extract_folder: p.Path = p.Path("ml-latest")

# Mac workaround (only if needed)
ssl._create_default_https_context = ssl._create_unverified_context

if not os.path.exists(extract_folder):
    print("Downloading MovieLens dataset...")
    urllib.request.urlretrieve(dataset_url, zip_file)
    print("Extracting...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        for member in zip_ref.namelist():
            filename = os.path.basename(member)
            if not filename:
                continue  # skip directories
            source = zip_ref.open(member)
            target_path = os.path.join(extract_folder, filename)
            with open(target_path, "wb") as target:
                target.write(source.read())    
    print("Done.")
else:
    print("Dataset already exists.")


Dataset already exists.


### Save User Data

In [14]:
RUN_HISTORY_FILENAME: str = "run_history.log"
LABEL_WIDTH = 40
VALUE_WIDTH = 20
written_model_headers: set[str] = set()
def log_run_header(user_count: int, movie_count: int, c_value: float, filename: str = RUN_HISTORY_FILENAME):
    timestamp: str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(filename, "a") as f:
        f.write("\n" + "=" * (LABEL_WIDTH + VALUE_WIDTH) + "\n")
        f.write(f"{'u_count:':<{LABEL_WIDTH}}{user_count:>{VALUE_WIDTH}}\n")
        f.write(f"{'m_count:':<{LABEL_WIDTH}}{movie_count:>{VALUE_WIDTH}}\n")
        f.write(f"{'c_value:':<{LABEL_WIDTH}}{c_value:>{VALUE_WIDTH}.4f}\n")
        f.write(f"{'Run at:':<{LABEL_WIDTH}}{timestamp:>{VALUE_WIDTH}}\n")
        f.write("-" * (LABEL_WIDTH + VALUE_WIDTH) + "\n")

def save_run_data(model_type: str, output_dict: dict, filename=RUN_HISTORY_FILENAME):
    with open(filename, "a") as f:
        header = {
                    "independent": "Independent Model", 
                    "ising": "Ising Model"
                 }.get(model_type.lower(), "Unknown Model")
        if model_type not in written_model_headers:
            f.write(f"{header}\n")
            written_model_headers.add(model_type)
        for key, value in output_dict.items():
            formatted_value = f"{value:.4f}" if isinstance(value, float) else str(value)
            f.write(f"{key:<{LABEL_WIDTH}}{formatted_value:>{VALUE_WIDTH}}\n")

log_run_header(USER_COUNT, MOVIE_COUNT, C_VALUE)

In [15]:
ratings = pd.read_csv('ml-latest/ratings.csv')

# Filter top users/movies
top_users = ratings['userId'].value_counts().head(USER_COUNT).index
top_movies = ratings['movieId'].value_counts().head(MOVIE_COUNT).index

filtered = ratings[(ratings['userId'].isin(top_users)) & (ratings['movieId'].isin(top_movies))]
pivot = filtered.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Binary matrix: 1 if liked (rating >= MINIMUM_RATING), else 0
X = (pivot >= MINIMUM_RATING).astype(int).to_numpy()

In [16]:
movies = pd.read_csv(os.path.join(extract_folder, 'movies.csv'))
id_to_title = dict(zip(movies['movieId'], movies['title']))

# Build a short label dictionary for visualization
# short = {i: id_to_title[mid] for i, mid in enumerate(pivot.columns)}

Xtr, Xte = train_test_split(X, test_size=0.2, random_state=42)
nMovies = Xtr.shape[1]

### Independent Model (trivial)

In [17]:
pXi = np.mean(Xtr, axis=0)
model0 = gm.GraphModel([gm.Factor([gm.Var(i, 2)], [1 - pXi[i], pXi[i]]) for i in range(nMovies)]) # type: ignore

ind_train_ll = np.mean([model0.logValue(x) for x in Xtr])
ind_test_ll = np.mean([model0.logValue(x) for x in Xte])

save_run_data("independent", {"- Log-Likelihood (Train)" : float(ind_train_ll), 
                              "- Log-Likelihood (Test)" : float(ind_test_ll)})

print("Independent model Train LL:", ind_train_ll)
print("Independent model Test  LL:", ind_test_ll)


Independent model Train LL: -155.09829868656539
Independent model Test  LL: -155.68010474922153


### Start of Ising Model

In [18]:
from sklearn.linear_model import LogisticRegression

nbrs, th_ij, th_i = [None] * nMovies, [None] * nMovies, np.zeros((nMovies,))
Xtmp = np.copy(Xtr)

for i in range(nMovies):
    Xtmp[:, i] = 0.
    lr = LogisticRegression(penalty='l1', C=C_VALUE, solver='liblinear').fit(Xtmp, Xtr[:, i])
    nbrs[i] = np.where(np.abs(lr.coef_) > 1e-6)[1]
    th_ij[i] = lr.coef_[0, nbrs[i]] / 2.
    th_i[i] = lr.intercept_ / 2.
    Xtmp[:, i] = Xtr[:, i]

average_connectivity = np.mean([len(nn) for nn in nbrs])
std_dev_average_connectivity = np.std([len(nn) for nn in nbrs])

save_run_data("ising", {"- Average Connectivity" : f"{average_connectivity:.4f} +/- {std_dev_average_connectivity:.4f}"})

print("Average connectivity at C =", C_VALUE, ":", average_connectivity)

Average connectivity at C = 0.045 : 10.456


In [19]:
factors = [gm.Factor(gm.Var(i, 2), [-t, t]).exp() for i, t in enumerate(th_i)] # type: ignore
for i in range(nMovies):
    for j, n in enumerate(nbrs[i]):
        scope = [gm.Var(i, 2), gm.Var(int(n), 2)]
        t = th_ij[i][j]
        factors.append(gm.Factor(scope, [[t, -t], [-t, t]]).exp()) # type: ignore

model1 = gm.GraphModel(factors)
model1.makeMinimal()


In [20]:
# Print mapping of movie indices to titles
# print("Movie Index to Title Mapping:")
# print("-" * 40)
# for var in model1.vars:
#     print(f"Movie {var.label}: {short[var.label]}")
# print("-" * 40)

# Draw graph with numeric labels
# short_labels = {var.label: var.label for var in model1.vars}
# gm.drawMarkovGraph(model1, labels=short_labels)


In [21]:
def conditional(factor, i, x):
    return factor.t[tuple(x[v] if v != i else slice(v.states) for v in factor.vars)]

def pseudolikelihood(model, X):
    LL = np.zeros(X.shape)
    for i in range(X.shape[1]):  # for each variable (movie)
        flist = model.factorsWith(i, copy=False)
        for j in range(X.shape[0]):  # for each data point (user)
            pXi = 1.
            for f in flist:
                pXi *= conditional(f, i, X[j])
            LL[j, i] = np.log(pXi[X[j, i]] / pXi.sum()) # type: ignore
    return LL.sum(1)


In [22]:
pseudolikelihood_tr: float = float(pseudolikelihood(model1, Xtr).mean())
pseudolikelihood_te: float = float(pseudolikelihood(model1, Xte).mean())

save_run_data("ising", {"- Pseudo-Likelihood (Train)" : pseudolikelihood_tr, 
                        "- Pseudo-Likelihood (Test)" : pseudolikelihood_te})

print("Pseudo-likelihood (Train):", pseudolikelihood_tr)
print("Pseudo-likelihood (Test):", pseudolikelihood_te)

Pseudo-likelihood (Train): -136.7481207341833
Pseudo-likelihood (Test): -142.41302892673804


In [23]:
def impute_missing(model, Xobs):
    m,n = Xobs.shape
    Xhat = np.copy(Xobs)
    for j in range(m):
        x_obs = {i:Xobs[j,i] for i in range(n) if Xobs[j,i] >= 0}
        x_unobs = [i for i in range(n) if Xobs[j,i] < 0]
        cond = gm.GraphModel([f.condition(x_obs) for f in model.factorsWithAny(x_unobs)])
        for x in cond.X:
            if x.states == 0:
                x.states = 1  # fix a bug in GraphModel behavior for missing vars...
        jt = pyGMs.wmb.JTree(cond, weights=1e-6) # 0: for maximization
        x_hat = jt.argmax()
        for i in x_unobs: 
            Xhat[j,i] = x_hat[i]
    return Xhat

In [24]:
# Create a copy of Xte to simulate missing values
Xte_missing = np.copy(Xte)

# Amount of test data that will be hidden
missing_proportion = 0.2

# Random seed for reproducibility
np.random.seed(42)

# Boolean mask where True means that position is missing and apply it to Xte_missing
mask = np.random.rand(*Xte.shape) < missing_proportion

# Set the selected entries to a missing indicator
Xte_missing[mask] = -1

In [25]:
# Slow!  (Constructing lots of conditional models...)
Xte_hat = impute_missing(model1, Xte_missing)

# Compare the imputed values (Xte_hat) with the original true values (Xte)
error_rate = np.mean(Xte_hat[mask] != Xte[mask]) * 100

save_run_data("ising", {"- Error Rate" : f"{error_rate:.4f}%"})

print(f"Error Rate: {error_rate:.4f}")

Error Rate: 25.0125


### Get most predictive movies of a single movie

In [26]:
movie_ids    = list(pivot.columns)           
movie_titles = [id_to_title[mid] for mid in movie_ids] 

# 2) build full J from nbrs & th_ij (from Cell 7)
J = np.zeros((nMovies, nMovies))
for i in range(nMovies):
    for k, j in enumerate(nbrs[i]):
        J[i, j] = th_ij[i][k]

# 3) symmetrize to get an undirected coupling matrix
J_sym = 0.5 * (J + J.T)

# sanity check
assert J_sym.shape == (nMovies, nMovies)

def get_predictive_movies(target_idx, J, movie_titles, top_k=10):
    """
    Return the top_k movies whose ratings (via |J[target, :]|)
    are most predictive of whether you'll like movie[target].
    
    Parameters
    ----------
    target_idx : int
        Index of the movie of interest.
    J : ndarray, shape (M, M)
        Learned Ising interaction matrix.
    movie_titles : list of str, length M
        Titles aligned with J’s rows/cols.
    top_k : int
        Number of top movies to return.
        
    Returns
    -------
    List of tuples (title, coupling_strength), sorted by |coupling| descending.
    """
    # extract couplings to all others
    w = J[target_idx].copy()
    # zero out self‐coupling
    w[target_idx] = 0
    # sort by absolute strength
    idx_sorted = np.argsort(np.abs(w))[::-1]
    top_idx = idx_sorted[:top_k]
    return [(movie_titles[i], w[i]) for i in top_idx]


In [40]:
target_idx = 0
short = {i: id_to_title[mid] for i, mid in enumerate(pivot.columns)}

top10 = get_predictive_movies(target_idx, J_sym, movie_titles, top_k=10)
print(f"Top 10 movies most predictive of liking {short[target_idx]!r}:")
for rank, (title, weight) in enumerate(top10, 1):
    sign = "➕" if weight > 0 else "➖"
    print(f"{rank:2d}. {title:40s} {sign} {abs(weight):.4f}")


Top 10 movies most predictive of liking 'Toy Story (1995)':
 1. Toy Story 2 (1999)                       ➕ 0.7183
 2. Finding Nemo (2003)                      ➕ 0.2473
 3. Bug's Life, A (1998)                     ➕ 0.1356
 4. Monsters, Inc. (2001)                    ➕ 0.0987
 5. Princess Bride, The (1987)               ➕ 0.0623
 6. Lion King, The (1994)                    ➕ 0.0526
 7. Up (2009)                                ➕ 0.0460
 8. Truman Show, The (1998)                  ➕ 0.0412
 9. Big (1988)                               ➕ 0.0318
10. Shawshank Redemption, The (1994)         ➕ 0.0271


### Given a list of movies recommend a single movie

In [47]:
def recommend_one_from_set(liked_idxs, movie_titles, J_sym):
    """
    Given a list of movies the user likes, recommend the single movie
    (not already in liked_titles) with the highest aggregate coupling.
    """
    # aggregate score for each candidate j: sum of J_sym[j, i] over liked i
    # we'll ignore negative influence by zeroing those out
    # shape (M,)
    agg = np.sum(np.where(J_sym[:, liked_idxs] > 0,
                          J_sym[:, liked_idxs],
                          0),
                 axis=1)
    
    # exclude already liked
    for i in liked_idxs:
        agg[i] = -np.inf
    
    # pick the best
    best_idx = np.argmax(agg)
    return movie_titles[best_idx], agg[best_idx]

In [58]:
liked = [114, 50, 18, 19, 36, 0]
rec_title, rec_score = recommend_one_from_set(liked, movie_titles, J_sym)
print("Because you like:")
for t in liked:
    print("   •", short[t])
print(f"\n→ we recommend: {rec_title!r}  (score = {rec_score:.4f})")

Because you like:
   • Jaws (1975)
   • Jurassic Park (1993)
   • Braveheart (1995)
   • Taxi Driver (1976)
   • Shawshank Redemption, The (1994)
   • Toy Story (1995)

→ we recommend: 'Toy Story 2 (1999)'  (score = 0.7407)


### Aggregate the most predictive movies across all 1,000 users and 250 movies

In [ ]:
from collections import Counter

target_idx = 1
short = {i: id_to_title[mid] for i, mid in enumerate(pivot.columns)}
aggregated = []

for i in range(MOVIE_COUNT):
    top10 = get_predictive_movies(i, J_sym, movie_titles, top_k=10)
    # print(f"Top 10 movies most predictive of liking {short[i]!r}:")
    for rank, (title, weight) in enumerate(top10, 1):
      aggregated.append(title)

print("\nTop 10 most frequently predictive movies:")
print("-" * 40)
for rank, (title, count) in enumerate(Counter(aggregated).most_common(10), 1):
    print(f"{rank:2d}. {title:40s} {count:3d} occurrences")



Top 10 most frequently predictive movies:
----------------------------------------
 1. Toy Story (1995)                          72 occurrences
 2. Jumanji (1995)                            46 occurrences
 3. Heat (1995)                               40 occurrences
 4. GoldenEye (1995)                          28 occurrences
 5. Casino (1995)                             26 occurrences
 6. Taxi Driver (1976)                        25 occurrences
 7. Big (1988)                                23 occurrences
 8. Sense and Sensibility (1995)              22 occurrences
 9. Citizen Kane (1941)                       21 occurrences
10. Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)  21 occurrences


In [45]:
for var in model1.vars:
    print(f"{var.label}: {short[var.label]}")
print("-" * 40)

0: Toy Story (1995)
1: Jumanji (1995)
2: Heat (1995)
3: GoldenEye (1995)
4: Casino (1995)
5: Sense and Sensibility (1995)
6: Ace Ventura: When Nature Calls (1995)
7: Get Shorty (1995)
8: Leaving Las Vegas (1995)
9: Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
10: Babe (1995)
11: Dead Man Walking (1995)
12: Clueless (1995)
13: Seven (a.k.a. Se7en) (1995)
14: Usual Suspects, The (1995)
15: Mr. Holland's Opus (1995)
16: Broken Arrow (1996)
17: Happy Gilmore (1996)
18: Braveheart (1995)
19: Taxi Driver (1976)
20: Birdcage, The (1996)
21: Apollo 13 (1995)
22: Batman Forever (1995)
23: Crimson Tide (1995)
24: Die Hard: With a Vengeance (1995)
25: Net, The (1995)
26: Waterworld (1995)
27: Clerks (1994)
28: Dumb & Dumber (Dumb and Dumber) (1994)
29: Interview with the Vampire: The Vampire Chronicles (1994)
30: Star Wars: Episode IV - A New Hope (1977)
31: Natural Born Killers (1994)
32: Outbreak (1995)
33: Léon: The Professional (a.k.a. The Professional) (Léon) (1994)
34: Pulp Fiction (1994)
35: 